In [66]:
import os 
from dotenv import load_dotenv
load_dotenv()
import gradio as gr
import openai
from openai import OpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain.prompts import MessagesPlaceholder

In [67]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"

In [68]:
# Load environment variables in a file called .env

load_dotenv()
if os.getenv("OPENAI_API_KEY") is None:
    print("OPENAI_API_KEY not found in .env file. Please set it.")
    exit()

MODEL = "gpt-4o-mini"
pdf_path = "KnowledgeBase/testEndometriosis.pdf" # <--- IMPORTANT: CHANGE THIS TO YOUR PDF FILE

# --- Imports for document handling and RAG ---
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


from langchain_core.messages import HumanMessage, AIMessage
from langchain.prompts import MessagesPlaceholder

# --- Global variable for the retriever ---
# This will hold our document retriever so the chat function can access it.
retriever = None

In [69]:
# --- 2. LOAD AND PROCESS THE PDF (Function) ---
# We put this in a function to keep the code clean.
def process_pdf(path):
    global retriever
    if not os.path.exists(path):
        print(f"Error: PDF file not found at {path}")
        return None

    print("PDF file found. Loading and processing...")
    loader = PyPDFLoader(path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    print(f"PDF split into {len(docs)} chunks.")

    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    print("Creating vector store...")
    vector_store = FAISS.from_documents(docs, embeddings)
    
    # Set the global retriever variable
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
    print("Vector store and retriever created successfully.")

# --- Process the initial PDF on startup ---
process_pdf(pdf_path)

PDF file found. Loading and processing...
PDF split into 314 chunks.
Creating vector store...
Vector store and retriever created successfully.


In [70]:

# --- 3. BUILD THE CONVERSATIONAL RAG CHAIN ---

### MODIFIED ### - This whole section is new and replaces the old `rag_chain`.
# This chain is more complex to handle chat history.

# a) First, a chain to condense the user's question and chat history into a standalone question.
# This is so the retriever gets a good query even if the user asks a follow-up question.
condense_question_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

llm = ChatOpenAI(model_name=MODEL, temperature=0.7)

# Chain to create the standalone question
condense_question_chain = condense_question_prompt | llm | StrOutputParser()


# b) The main chain that answers the question using the retrieved context.
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "ou are a helpful and knowledgeable assistant for medical students. Answer the user’s question using only the information provided in the context below, which is sourced from official medical guidelines and references. If the answer is not clearly supported by the context, respond with “I don’t know based on the provided information.” Do not make assumptions or provide information beyond what is stated in the context.\n\nContext:\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

# This function determines which question to use for retrieval (original or condensed)
def get_retrieval_question(input_dict):
    if input_dict.get("chat_history"):
        return condense_question_chain
    else:
        return input_dict["question"]

# c) The final conversational RAG chain.
# It chains together the question condensation, document retrieval, and final answer generation.
conversational_rag_chain = (
    RunnablePassthrough.assign(
        context=get_retrieval_question | retriever 
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [71]:
import sys
!{sys.executable} -m pip install faiss-cpu


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [72]:
# --- 3. CREATE EMBEDDINGS AND VECTOR STORE ---
# Import components for embeddings and vector storage
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# Create an embeddings model instance. This will convert our text chunks into numerical vectors.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create a FAISS vector store from the document chunks and their embeddings.
# This store allows for very fast similarity searches.
if 'docs' in locals() and docs:
    print("Creating vector store...")
    vector_store = FAISS.from_documents(docs, embeddings)
    print("Vector store created successfully.")
    # Create a retriever from the vector store. The retriever's job is to fetch relevant documents.
    retriever = vector_store.as_retriever()
else:
    print("No documents were loaded. Skipping vector store creation.")
    retriever = None

Creating vector store...
Vector store created successfully.


In [ ]:
# --- 4. CREATE AND LAUNCH THE GRADIO INTERFACE ---

### MODIFIED ### - The chat function now handles history.
def chat_function(message, history):
    """The function that powers the Gradio chat interface."""
    if retriever is None:
        return "Sorry, the document could not be loaded. Please check the PDF file path and restart."

    # Convert Gradio's history format to LangChain's message format
    # Gradio history is a list of lists: [['user_msg_1', 'bot_msg_1'], ['user_msg_2', 'bot_msg_2']]
    chat_history_for_chain = []
    for user_msg, ai_msg in history:
        chat_history_for_chain.append(HumanMessage(content=user_msg))
        chat_history_for_chain.append(AIMessage(content=ai_msg))

    # Invoke the conversational RAG chain with the message and history
    response = conversational_rag_chain.invoke({
        "question": message,
        "chat_history": chat_history_for_chain
    })
    
    return response

print("\nLaunching Gradio Interface...")
print("Open the URL in your browser to chat with your RAG agent.")

ui = gr.ChatInterface(
    fn=chat_function,
    title="Antenatal Care AI",
    description="Ask any question the topic. The agent remembers your conversation.",
    examples=[
    ["What are the recommended antenatal visits during pregnancy?"],
    ["How is preeclampsia identified and managed antenatally?"],
    ["What supplements should be given during antenatal care?"],
    ["What are the key components of a first-trimester check-up?"]
]
)
ui.launch(share=True)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(



Launching Gradio Interface...
Open the URL in your browser to chat with your RAG agent.
* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
